# DIY Covid-19 Dashboard - a comparison of hospital admissions with ventilator bed occupancy

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("admissions_with_covidOccupiedMVBeds.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

## Table of admissions and occupied ventilator beds

This table shows admissions and ventilator bed occupancy for the earliest and latest 5 days from available data  

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['admissions', 'occupied ventilator beds'])
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['admissions', 'occupied ventilator beds']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(df.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                df.loc[date, column]=value

    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)
    return df
# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting
df

,admissions,occupied ventilator beds
2020-03-01,3.0,0.0
2020-03-02,1.0,0.0
2020-03-03,0.0,0.0
2020-03-04,0.0,0.0
2020-03-05,1.0,0.0
...,...,...
2022-10-15,900.0,221.0
2022-10-16,1206.0,217.0
2022-10-17,1121.0,207.0
2022-10-18,0.0,200.0


## Download current data

Click the button below to download the latest data available for admissions and ventilator beds occupancy

In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    # put code for polling the API here
    filters = [
    'areaType=nation'  
    ]
    structure = {
        "date": "date",
        "admissions": "newAdmissions",
        "occupied ventilator beds": "covidOccupiedMVBeds"

    }
    api = Cov19API(filters=filters, structure=structure)
    jsondata=api.get_json()
    return jsondata 

In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Fetch Data', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Fetch Data', icon='download', style=ButtonStyle(), tooltip='Click to …

## Graphs and Analysis

This graph shows admissisons and ventilator bed occupancy from the earliest to latest available data. The widget enables visualisation of data in as either linear or log graphs.

In [7]:
series=wdg.SelectMultiple(
    options=['admissions', 'occupied ventilator beds'],
    value=['admissions', 'occupied ventilator beds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

**2022 Nasim Mohamed Ismail**. *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*